In [15]:
import cvxpy as cp
import numpy as np
from timeit import default_timer as timer
from itertools import product

In [16]:
cp.installed_solvers()

['CLARABEL',
 'CVXOPT',
 'ECOS',
 'ECOS_BB',
 'GLPK',
 'GLPK_MI',
 'OSQP',
 'SCIPY',
 'SCS']

In [17]:
def col(matrix, col):
    return matrix[:, col - 1]

def row(matrix, row):
    return matrix[row-1,:]

def idx(matrix, axis1, axis2=None):
    if axis2 is None:
        return matrix[axis1 - 1]
    return matrix[axis1 - 1][axis2 - 1]

def vec(number, amount):
    return np.repeat(number, amount)

def irange(start, end):
    return range(start, end + 1)

# Constants

$$
\text{cost} \in \mathbb{N}^{33}
$$

$$
\text{time} \in \mathbb{N}^{33}
$$

$$
\text{profit} \in \mathbb{N}^{33}
$$

$$
\text{area} \in \mathbb{N}^{33}
$$

$$
\text{button\_income} \in \mathbb{N}^{9}
$$


In [18]:
cost = np.array([2, 10, 5, 8, 7, 4, 2, 2, 2, 6, 2, 1, 7, 10, 4, 7, 1, 5, 10, 4, 1, 1, 1, 3, 2, 2, 3, 7, 3, 5, 3, 3, 0])
time = np.array([1, 4, 3, 6, 6, 2, 1, 3, 2, 5, 3, 2, 2, 5, 6, 4, 5, 4, 3, 2, 4, 3, 2, 1, 2, 2, 2, 1, 3, 5, 6, 4, 3])
profit = np.array([0, 3, 1, 3, 3, 0, 0, 0, 0, 2, 1, 0, 2, 3, 2, 2, 1, 2, 2, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 2, 2, 1, 1])
area = np.array([2, 5, 8, 6, 4, 6, 6, 7, 5, 4, 5, 6, 6, 6, 4, 6, 6, 5, 5, 4, 7, 3, 5, 3, 4, 3, 4, 5, 4, 5, 6, 5, 6])
button_income = np.array([4, 10, 16, 22, 28, 34, 40, 46, 52])

# Variables

$$X \in \{0,1\}^{33\times9}$$
$$Y \in \{0,1\}^{33\times9}$$
$$\psi \in \mathbb{N}^{9}$$
$$\phi \in \{0,1\}^{9}$$
$$\xi \in \mathbb{N}^{9}$$
$$\lambda \in \{0,1\}$$
$$\omega \in \{0,1\}^{33}$$


In [19]:
X = cp.Variable((33, 9), boolean=True)
Y = cp.Variable((33, 9), boolean=True)
ψ = cp.Variable(9, integer=True)
φ = cp.Variable(9, boolean=True)
ξ = cp.Variable(8, integer=True)
λ = cp.Variable(1, boolean=True)
ω = cp.Variable(33, boolean=True)

# Constraints

## Area Constraints

$$a^\mathsf{T}X\vec{e_9} \le 81$$
$$a^\mathsf{T}X\vec{e_9} - 76 + 1 \le 1000\lambda$$
$$76\lambda \le a^\mathsf{T}X\vec{e_9}$$

## Time Constraints

$$\left(X\vec{e_1}\ \phantom{ - X\vec{e}_{j-1} } - Y\vec{e_1} \right)^\mathsf{T}t + \psi_1 \le b_1 $$
$$\ \xi*{j-1} + \left(X\vec{e_j} - X\vec{e}*{j-1} - Y\vec{e*j} \right)^\mathsf{T}t + \psi_j \le b_j \qquad j=2,\dots,9$$
$$b_j \le \xi_j \qquad j=1,\dots,8$$
$$t^\mathsf{T}X\vec{e_1} + \psi_1 \le \xi_1 $$
$$\xi*{j-1} + t^\mathsf{T}\left(X\vec{e*j} - X\vec{e}*{j-1}\right) + \psi*j \le \xi_j \qquad j=2,\dots,8$$
$$\left(Y\vec{e_j}\right)^\mathsf{T}\vec{1} \le 1 \qquad j=1,\dots,9 $$
$$Y*{i,j} \le X*{i,j} \qquad i=1,\dots,33\quad j=1\phantom{,\dots,9} $$
$$Y*{i,j} \le X*{i,j} - X*{i,j-1} \qquad i=1,\dots,33\quad j=2,\dots,9$$

## Button Balance >= 0 Constraints

$$5 + \sum_{k=1}^{j-1}\left( p^\mathsf{T}X\vec{e_k} + \psi_k \right) \ge c^\mathsf{T}X\vec{e_j} \qquad j=1,\dots,9$$

## Phi Constraints

$$\phi_j \le \psi_j \qquad j=1,\dots,9$$
$$\phi_j + \left(Y\vec{e_j}\right)^\mathsf{T}\vec{1} \le 1 \qquad j=1,\dots,9$$

## Further Constraints

$$X_{i,j} \le X_{i,j+1} \qquad  i=1,\dots,33\quad j=1,\dots,8$$
$$\psi_j \ge b_j - t^\mathsf{T}X\vec{e_j} - \sum_{k=1}^{j-1} \psi_k \qquad j=1,\dots,9$$

## Omega Multiplication

$$\omega_i \le X_{i,9} \qquad i=1,\dots,33$$
$$\omega^\mathsf{T}\vec{1} \le 33 \cdot \lambda$$
$$X_{i,9} + \lambda - 1 \le \omega_i \qquad i=1,\dots,33$$

## Variable Constraints

$$\psi \ge 0$$
$$\xi \ge 0$$


In [20]:
def define_constraints(X, Y, ψ, φ, ξ, λ, ω, check=False):
    constraints = [
        # Area Constraints
        area @ col(X, 9) <= 81,
        area @ col(X, 9) - 76 + 1 <= 1000 * λ,
        76 * λ <= area @ col(X, 9),

        # Time Constraints
        (col(X, 1) - col(Y, 1)) @ time + idx(ψ, 1) - idx(φ, 1) <= idx(button_income, 1),
*(      idx(ξ, j-1) + (col(X, j) - col(X, j-1) - col(Y, j)) @ time + idx(ψ, j) - idx(φ, j) <= idx(button_income, j)              for j in irange(2,9) ),
*(      idx(ξ, j) >= idx(button_income, j) + 1                                                                                   for j in irange(1,8) ),
        idx(ξ, 1) >= time @ col(X, 1) + idx(ψ, 1),
*(      idx(ξ, j) >= idx(ξ, j-1) + time @ (col(X, j) - col(X, j-1)) + idx(ψ, j)                                                  for j in irange(2,8) ),
*(      col(Y, j) @ vec(1, 33) <= 1                                                                                              for j in irange(1,9) ),
*(      idx(Y, i, 1) <= idx(X, i, 1)                                                                                           for i in irange(1, 33) ),
*(      idx(Y, i, j) <= idx(X, i, j) - idx(X, i, j-1)                                               for (i, j) in product(irange(1, 33), irange(2,9)) ),

        # Button Balance >= 0 Constraints
*(      5 + sum(profit @ col(X, k) for k in irange(1, j-1)) + sum(idx(ψ, k) for k in irange(1, j)) >= cost @ col(X, j)           for j in irange(1,9) ),

        # Phi Constraints
*(      idx(φ, j) <= idx(ψ, j)                                                                                                   for j in irange(1,9) ),
*(      idx(φ, j) + col(Y, j) @ vec(1, 33) <= 1                                                                                  for j in irange(1,9) ),

        # Further Constraints
*(      idx(X, i, j) <= idx(X, i, j + 1)                                                             for (i, j) in product(irange(1,33), irange(1,8)) ),
*(      idx(ψ, j) >= idx(button_income, j) - time @ col(X, j) - sum(idx(ψ, k) for k in irange(1,j-1))                            for j in irange(1,9) ),

        # Omega Multiplication
*(      idx(ω, i) >= idx(X, i, 9)                                                                                               for i in irange(1,33) ),
        ω @ vec(1, 33) <= 33 * λ,
*(      idx(ω, i) >= idx(X, i, 9) + λ - 1                                                                                       for i in irange(1,33) ),
    ]

    if check == False:
        constraints.extend([
            # Variable Constraints
            0 <= ψ,
            0 <= ξ,
        ])

    return constraints

In [21]:
constraints = define_constraints(X, Y, ψ, φ, ξ, λ, ω)

In [22]:
objective = cp.Maximize(
    5 + 7 - 81 * 2
    + profit @ X @ vec(1, 9)
    + 2 * 81 * λ
    + 2 * area @ col(X, 9)
    - 2 * area @ ω
    - cost @ col(X, 9)
    + ψ @ vec(1,9)
)

In [23]:
problem = cp.Problem(objective, constraints)
fast = True

# https://www.cvxpy.org/tutorial/advanced/index.html Solver args

kwargs = None,
if fast:
    kwargs = {
        'max_iters': 100, # maximum number of iterations (default: 100).
        'abstol': 1e-7, # absolute accuracy (default: 1e-7).
        'reltol': 1e-6, # relative accuracy (default: 1e-6).
        'feastol': 1e-7, # tolerance for feasibility conditions (default: 1e-7).
    }
else:
    kwargs = {
        'max_iters': 500,
        'abstol': 1e-20,
        'reltol': 1e-20,
        'feastol': 1e-20,
    }

start = timer()
result = problem.solve(verbose=True, **kwargs)
end = timer()

print(f"Problem status: {problem.status}")
print(problem.solver_stats)
print(result)
print(f'Solver took {end - start}s')

                                     CVXPY                                     
                                     v1.4.2                                    
(CVXPY) Mar 30 11:28:31 AM: Your problem has 654 variables, 703 constraints, and 0 parameters.
(CVXPY) Mar 30 11:28:31 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Mar 30 11:28:31 AM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Mar 30 11:28:31 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Mar 30 11:28:31 AM: Your problem is compiled with the CPP canonicalization backend.


-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Mar 30 11:28:31 AM: Compiling problem (target solver=GLPK_MI).
(CVXPY) Mar 30 11:28:31 AM: Reduction chain: FlipObjective -> Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffing -> GLPK_MI
(CVXPY) Mar 30 11:28:31 AM: Applying reduction FlipObjective
(CVXPY) Mar 30 11:28:31 AM: Applying reduction Dcp2Cone
(CVXPY) Mar 30 11:28:31 AM: Applying reduction CvxAttr2Constr
(CVXPY) Mar 30 11:28:31 AM: Applying reduction ConeMatrixStuffing
(CVXPY) Mar 30 11:28:32 AM: Applying reduction GLPK_MI
(CVXPY) Mar 30 11:28:32 AM: Finished problem compilation (took 6.326e-01 seconds).
-------------------------------------------------------------------------------
                                Numerical solver                               
------------------------

In [24]:
# problem.get_problem_data(solver=cp.GLPK_MI)

In [25]:
np.set_printoptions(
    #edgeitems=30,
    linewidth=100000,
    #formatter=dict(float=λa x: "%.3g" % x)
)

print(f"Value = {result}")
print(f"   Start    = {5}");
print(f"   SpecTile = {7}");
print(f"   Profit   = {profit @ X.value @ vec(1,9)}")
print(f"   Std Area = {-2 * 81}");
print(f"   Area     = {2 * 81 * λ.value.item() + 2 * area @ col(X.value, 9) - 2 * area @ ω.value}")
print(f"       Area1    = {2 * 81 * λ.value.item()}")
print(f"       Area2    = {2 * area @ col(X.value, 9)}")
print(f"       Area3    = {- 2 * area @ ω.value}")
print(f"   Cost     = {- cost @ col(X.value, 9)}")
print(f"   Walk     = {ψ.value @ vec(1,9)}")

print(f"patches             = {np.argwhere(col(X.value, 9)).squeeze()}")
area_cost = col(X.value, 9) * area;
print(f"patches area        = {area_cost[area_cost > 0]} (sum = {area_cost[area_cost > 0].sum()})")
time_cost = col(X.value, 9) * time;
print(f"patches time cost   = {time_cost[time_cost > 0]} (sum = {time_cost[time_cost > 0].sum()})")
button_cost = col(X.value, 9) * cost;
print(f"patches button cost = {button_cost[button_cost > 0]} (sum = {button_cost[button_cost > 0].sum()})")

print(f"patches [1]         = {np.argwhere(col(X.value, 1)).squeeze()}")
print(f"patches [2]         = {np.argwhere(col(X.value, 2)).squeeze()}")
print(f"patches [3]         = {np.argwhere(col(X.value, 3)).squeeze()}")
print(f"patches [4]         = {np.argwhere(col(X.value, 4)).squeeze()}")
print(f"patches [5]         = {np.argwhere(col(X.value, 5)).squeeze()}")
print(f"patches [6]         = {np.argwhere(col(X.value, 6)).squeeze()}")
print(f"patches [7]         = {np.argwhere(col(X.value, 7)).squeeze()}")
print(f"patches [8]         = {np.argwhere(col(X.value, 8)).squeeze()}")
print(f"patches [9]         = {np.argwhere(col(X.value, 9)).squeeze()}")

print(f"X[9] = {col(X.value, 9)}")
print(f"X^T =\n{X.value.T}")
print(f"Y^T =\n{Y.value.T}")
print(f"ψ = {ψ.value}")
print(f"φ = {φ.value}")
print(f"ξ = {ξ.value}")
print(f"λ = {λ.value}")
print(f"ω = {ω.value}")
print("\ngiven constants")
print(f"b = {button_income}")
print(f"t = {time}")
print(f"p = {profit}")
print(f"a = {area}")
print(f"c = {cost}")

Value = 85.0
   Start    = 5
   SpecTile = 7
   Profit   = 117.0
   Std Area = -162
   Area     = 162.0
       Area1    = 162.0
       Area2    = 154.0
       Area3    = -154.0
   Cost     = -51.0
   Walk     = 7.0
patches             = [ 1  3  4  6 10 11 12 16 17 20 22 26 30 32]
patches area        = [5. 6. 4. 6. 5. 6. 6. 6. 5. 7. 5. 4. 6. 6.] (sum = 77.0)
patches time cost   = [4. 6. 6. 1. 3. 2. 2. 5. 4. 4. 2. 2. 6. 3.] (sum = 50.0)
patches button cost = [10.  8.  7.  2.  2.  1.  7.  1.  5.  1.  1.  3.  3.] (sum = 51.0)
patches [1]         = [17 32]
patches [2]         = [10 17 20 32]
patches [3]         = [ 4 10 17 20 32]
patches [4]         = [ 3  4 10 17 20 32]
patches [5]         = [ 1  3  4 10 17 20 32]
patches [6]         = [ 1  3  4 10 12 17 20 30 32]
patches [7]         = [ 1  3  4 10 12 17 20 26 30 32]
patches [8]         = [ 1  3  4 10 11 12 17 20 26 30 32]
patches [9]         = [ 1  3  4  6 10 11 12 16 17 20 22 26 30 32]
X[9] = [0. 1. 0. 1. 1. 0. 1. 0. 0. 0. 1. 1. 1. 0. 0.

# Solution Test


In [26]:
X = np.array([
   #                     1,1,1,1,1,1,1,1,1,1,2,2,2,2,2,2,2,2,2,2,3,3,3
   # 0,1,2,3,4,5,6,7,8,9,0,1,2,3,4,5,6,7,8,9,0,1,2,3,4,5,6,7,8,9,0,1,2
    [0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1],
    [0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1],
    [0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1],
    [0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1],
    [0,1,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1],
    [0,1,1,1,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1],
    [0,1,1,1,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,1,0,1],
    [0,1,1,1,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,1,0,1],
    [0,1,1,1,0,0,1,0,0,0,1,1,1,0,0,0,1,1,0,0,1,0,0,0,0,0,1,0,0,0,1,0,1],
], dtype=np.int64).T
Y = np.array([
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0],
    [0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
], dtype=np.int64).T
ψ = np.array([2,2,0,0,0,0,3,3,2], dtype=np.int64)
φ = np.array([0,0,0,0,0,0,0,1,0], dtype=np.int64)
ξ = np.array([7,13,22,28,32,37,44,47], dtype=np.int64)
λ = np.array([1], dtype=np.int64);
ω = col(X, 9) * λ

In [27]:
def check_constraints(X, Y, ψ, φ, ξ, λ, ω):
    constraints = define_constraints(X, Y, ψ, φ, ξ, λ, ω)
    constraints = [item for sublist in constraints for item in (sublist if isinstance(sublist, list) or isinstance(sublist, np.ndarray) else [sublist])]

    print(f"Constraints of length {len(constraints)}")

    if np.all(constraints[0:3]) == False:
        print(f'Area Constraints (False): {constraints[0:3]}')

    if np.all(constraints[3:12]) == False:
        print(f'Time Constraints 1 (False): {constraints[3:12]}')

    if np.all(constraints[12:20]) == False:
        print(f'Time Constraints 2 (False): {constraints[12:20]}')

    if np.all(constraints[20:28]) == False:
        print(f'Time Constraints 3 (False): {constraints[20:28]}')

    if np.all(constraints[28:37]) == False:
        print(f'Time Constraints 4 (False): {constraints[28:37]}')

    if np.all(constraints[37:334]) == False:
        print(f'Time Constraints 5 (False): {constraints[37:334]}')

    if np.all(constraints[334:343]) == False:
        print(f'Button Balance >= 0 Constraints (False): {constraints[334:343]}')

    if np.all(constraints[343:607]) == False:
        print(f'Further Constraints 1 (False): {constraints[343:607]}')

    if np.all(constraints[607:616]) == False:
        print(f'Further Constraints 2 (False): {constraints[607:616]}')

    if np.all(constraints[616:649]) == False:
        print(f'Omega Multiplication 1 (False): {constraints[616:649]}')

    if np.all(constraints[649:650]) == False:
        print(f'Omega Multiplication 2 (False): {constraints[649:650]}')

    if np.all(constraints[650:683]) == False:
        print(f'Omega Multiplication 3 (False): {constraints[650:683]}')

    if np.all(constraints[683:692]) == False:
        print(f'NEW φ 1 (False): {constraints[683:692]}')

    if np.all(constraints[692:701]) == False:
        print(f'NEW φ 2 (False): {constraints[692:701]}')

    return np.array(constraints)

In [28]:
constraints = check_constraints(X, Y, ψ, φ, ξ, λ, ω)

Constraints of length 718
